Giải capcha

In [8]:
from capmonstercloudclient import CapMonsterClient, ClientOptions
from capmonstercloudclient.requests import RecaptchaV2ProxylessRequest

API_KEY = "b238f538e55b7deb0da93267f61d8763"
WEBSITE_URL = 'https://af.uppromote.com/solar-power-store-canada/login'
WEBSITE_KEY = '6LcfFqkaAAAAAODkHHT2DLE7UBeSbf7kVCdBkTQE'
async def solve_captcha():
    client_options = ClientOptions(api_key=API_KEY)
    cap_monster_client = CapMonsterClient(options=client_options)
    recaptcha2request = RecaptchaV2ProxylessRequest(
        websiteUrl=WEBSITE_URL,
        websiteKey=WEBSITE_KEY
    )
    return await cap_monster_client.solve_captcha(recaptcha2request)


Login


In [9]:
import requests
import re

def extract_user_id(html_content):
    """
    Extracts the user ID from HTML content containing a JavaScript redirect.
    
    Args:
    - html_content (bytes): The HTML content in bytes.

    Returns:
    - str or None: The extracted user ID if found, or None if not found.
    """
    # Convert bytes to string
    html_str = html_content.decode()

    # Use regular expression to extract the user ID
    match = re.search(r'id=([a-f\d]+)', html_str)
    if match:
        return match.group(1)
    else:
        return None

def login_and_get_id(login_url, payload, headers): 
    print(payload)
    response = requests.post(login_url, data=payload, headers=headers)
    
    if response.status_code == 200:
        print(response.cookies)
        id = extract_user_id(response.content)
        if id:
            return id
        else:
            print("Không tìm thấy token.")
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


In [10]:
url = 'https://af.uppromote.com/solar-power-store-canada/login_aff'

res = await solve_captcha()
payload = {
    "_token": "l8E3hNPResTf5EPi9IuKX6U96dMwC5tq2dcFd8d2",
    "shop_id": "80375",
    "email": "teamasmads@gmail.com",
    "password": "2N*G5k$7ux5j2!F",
    "g-recaptcha-response": res["gRecaptchaResponse"]
}
headers = {
    
}

id = login_and_get_id(url, payload, headers)
print(f"id= {id}")


{'_token': 'l8E3hNPResTf5EPi9IuKX6U96dMwC5tq2dcFd8d2', 'shop_id': '80375', 'email': 'teamasmads@gmail.com', 'password': '2N*G5k$7ux5j2!F', 'g-recaptcha-response': '03AFcWeA6WJn16Jm8eym-T3sbSQcMve2ShfMbETNL_-hzvIcCpRfFSGBtb6BsbOkbDoK49LosiwU3nIOmzCCVuL7XsCB2WDWL40vSHA0UKuTt5VY7O6-WW15HKhp1C4J6w3mlOs1l5hCWVsBNaxyj2CJRPVlIRgDnHJcax3OvZyQmCh7ISwDR3wdPWikNdpc0ZI00bSBTQuir06mwyxKS5tOCUFeRl1Fa1RWtMyUNcugOe-3XdeA05DdlULAvaCrw2sGbyYwtbzd_1P80qyv9wojgMjyqbFBTdOEQaHx4rsD73Ls3QoJgze2hNIh8h4E5Ruhmz6RlIy-yoWBJgYKzjAu3IZ-q9uYbi4qoUHN2fUBAnE0Bv-S6zlwBi76sbjnr7COhl5y90svTVmpGU1nlk2gydtSJtqTudzbw6YSPYWwje6guyKxpZIcwI4P3R4rVRbdw2ufHuv6o1oHAHqFgje6Wqu5OYmp8C9GULEOEQ6OB_p5Q5q5t95vVKwBfgi38BtSB5wSLfHjuPqR9igizIfD0eZpsLM8lHbQkuLrCDyAgvlL4vzqu27oJ1PAnIf3ixxiDbIeNOr86otSMTJUPPWRNgUkRzt_CtwInBkSCx-mKTP5XksbrFHGA'}
<RequestsCookieJar[<Cookie XSRF-TOKEN=eyJpdiI6Ilc4eE1aRkdtR2VaSzJyOEZheXVQNEE9PSIsInZhbHVlIjoiTDNEbnV2Z0ZXZDVnbkx0bEI5ZUo0TEczVzMwRGhidVZTYVFWWG1mbGlcL25ob1owY2ZaYU1pOWwxTlFoT2VnckYiLCJtYWMiOiIxZWEzZj